In [116]:
import pandas as pd
import re
from lxml import etree
import shutil
import os
import math
import copy

In [117]:
def load_base_value(file_path):

    # Check if the file exists
    if os.path.exists(file_path):
        
        # File exists, load the variable from the file
        if os.path.getsize(file_path) > 0:
                    # File is not empty, load the variable from the file
            with open(file_path, 'r') as file:
                
                variable_loaded = file.read().strip()
#                 print('loading')
#                 print(variable_loaded)
        else:
            # File exists but is empty, initialize with a default value
            print('---')
            variable_loaded = "0195:SGUEN1"
    else:
        # File does not exist, create the file and write the variable to it
#         print('+++++')
        variable_to_store = "0195:SGUEN1"
        with open(file_path, 'w') as file:
            file.write(variable_to_store)
        variable_loaded = variable_to_store

    return variable_loaded
def save_base_value(variable_to_store,file_path):
    
    # Write the variable to the file
    with open(file_path, 'w') as file:
        file.write(variable_to_store)

def increment_value(value,file_path):
    pattern = r'SGUEN(\d+)'
    
    match = re.search(pattern, value)
    id_val = int(match.group(1))
    print(f'id val is {id_val}')
    id_val+=1
    if os.path.exists(file_path):
        with open(file_path, 'w') as file:
            variable_to_store = '0195:SGUEN' + str(id_val)
            print('writing')
            print(value)
            print('saving to new sender identifier to store')
            file.write(variable_to_store)
        

    else:
        print('load base value first')
    
    
def reset_value(file_path,value):
#     file_path = "stored_variable.txt"
    variable_to_store = value
    with open(file_path, 'w') as file:
        file.write(variable_to_store)
        
def get_element_xpath(element):
    path = [element.tag.split('}')[-1]]
    
    for parent in element.iterancestors():
        parent_tag = parent.tag.split('}')[-1]  
        path.insert(0, parent_tag)
    return ' / '.join(path)
def read_xml_file(file_path_xml):
    result_xml_dict={}
    tree = etree.parse(file_path_xml)
    target_root = tree.getroot()
#     pattern = r'\{.*?\}'
    for element in target_root.iter():
        if element.attrib:
            xpath_base = get_element_xpath(element)
            for e,val in element.attrib.items():
                xpath = xpath_base + ' / ' + '@' + e
#                 print(f'-->{xpath}')
                result_xml_dict[xpath] = tree.getpath(element)
        else:
            xpath = get_element_xpath(element)
            result_xml_dict[xpath] = tree.getpath(element)
            
    return result_xml_dict
        
def get_element_full_path(tree,element):
    return tree.getpath(element)

def read_xml_file_2(file_path_xml):
    tree = etree.parse(file_path_xml)
    target_root = tree.getroot()
    for element in target_root.iter():
        xpath = tree.getpath(element)
        print(f"XPath: {xpath}")

In [118]:
os.getcwd()

'/Users/cheongray/iras_api_make_xml/combine_all_copy'

In [119]:
#set up folders to store test_results and generated xmls
user_root = os.getcwd()
root_path = os.path.join(user_root,'iras_api_clean_1')



output_folder = os.path.join(root_path,'1_Input_Validation')
# output_folder = '/Users/cheongray/iras_api_clean_1/1_Input_Validation'

os.makedirs(output_folder, exist_ok=True)  # Create folder if it doesn't exist

output_file = os.path.join(output_folder, 'df_full.csv')
output_file_original = os.path.join(output_folder, 'df_full_back_up.csv')
if os.path.exists(output_file):
    os.remove(output_file)
if os.path.exists(output_file_original):
    os.remove(output_file_original)


In [121]:
#create AP_SP folder and move file into it
ap_sp_file = os.path.join(user_root,'AP_SP_FINAL_5.xlsx')
ap_sp_folder_file_path = os.path.join(root_path,'AP_SP_files')
os.makedirs(ap_sp_folder_file_path,exist_ok=True)
AP_SP_file_path_final = os.path.join(ap_sp_folder_file_path,'AP_SP_FINAL_5.xlsx')
if os.path.exists(ap_sp_folder_file_path):
    if os.path.exists(AP_SP_file_path_final):
        print(f'{AP_SP_file_path_final} already there')
    else:
        print(f'moving from {ap_sp_file} to {AP_SP_file_path_final} ')
        shutil.move(ap_sp_file,AP_SP_file_path_final)
    
    
    


/Users/cheongray/iras_api_make_xml/combine_all_copy/iras_api_clean_1/AP_SP_files/AP_SP_FINAL_5.xlsx already there


In [122]:
#Read in excel file
file_path = os.path.join(root_path,'AP_SP_files','AP_SP_FINAL_5.xlsx')
sheet_name = '1_Input Validation'#specific page being read

path_to_sample_xml_files = os.path.join(user_root,'sample_xml_files')

df_s = pd.read_excel(file_path, sheet_name=sheet_name)
df_s_temp = df_s.copy()
df_s_temp['Test Case ID'] = df_s_temp.index.astype(float)

df_s_temp['Test Case ID'] = df_s_temp['Test Case ID'] + 1
df_s_temp.to_csv(output_file, index=False)
df_s_temp.to_csv(output_file_original, index=False)


In [123]:
grouped = df_s_temp.groupby('Rule ID')
result_dict={}
for rule_id,group_data in grouped:
    test_cases = group_data.groupby('Test Case ID')
    test_cases_for_rule = []
    for test_case_id,test_data in test_cases:
        temp_dict = {}
        test_data_values = {
            'Field' : test_data['Field'].tolist(),
            'Field Value' : test_data['Field Value'].tolist(),
            'Test Case Description' : test_data['Test Case Description'].tolist(),
            'Expected Result' : test_data['Expected Result'].tolist(),
            'Standard' : test_data['Standard'].tolist(),
            'Document Type' : test_data['Document Type'].tolist(),
            'BIS/PINT' : test_data['BIS/PINT'].tolist(),
            'Sample' : test_data['Sample'].tolist(),
            'Test Case ID' : test_data['Test Case ID'].tolist(),
            'Rule ID' : test_data['Rule ID'].tolist()
        }
        temp_dict[test_case_id] = test_data_values
        test_cases_for_rule.append(temp_dict)
    result_dict[rule_id] = test_cases_for_rule
        
    

In [124]:
result_dict

{'IRASC5-001': [{1.0: {'Field': ['StandardBusinessDocument / StandardBusinessDocumentHeader / Sender / Identifier'],
    'Field Value': ['0195:SGUEN200012345A'],
    'Test Case Description': ['Field exists with valid value'],
    'Expected Result': [200],
    'Standard': ['PEPPOL'],
    'Document Type': ['Invoice'],
    'BIS/PINT': ['BIS'],
    'Sample': ['Sample 1'],
    'Test Case ID': [1.0],
    'Rule ID': ['IRASC5-001']}},
  {2.0: {'Field': ['StandardBusinessDocument / StandardBusinessDocumentHeader / Sender / Identifier'],
    'Field Value': ['0195:SGUEN'],
    'Test Case Description': ['Field exists with invalid value'],
    'Expected Result': [200],
    'Standard': ['PEPPOL'],
    'Document Type': ['Invoice'],
    'BIS/PINT': ['BIS'],
    'Sample': ['Sample 1'],
    'Test Case ID': [2.0],
    'Rule ID': ['IRASC5-001']}},
  {3.0: {'Field': ['StandardBusinessDocument / StandardBusinessDocumentHeader / Sender / Identifier'],
    'Field Value': [nan],
    'Test Case Description': ['

In [125]:
for key in result_dict:
    print(key)

IRASC5-001
IRASC5-002
IRASC5-003
IRASC5-004
IRASC5-005
IRASC5-006
IRASC5-007
IRASC5-008
IRASC5-009
IRASC5-010
IRASC5-012
IRASC5-017
IRASC5-018
IRASC5-020
IRASC5-021
IRASC5-022
IRASC5-024
IRASC5-025
IRASC5-026
IRASC5-027
IRASC5-028
IRASC5-029
IRASC5-030
IRASC5-031
IRASC5-033
IRASC5-034
IRASC5-035
IRASC5-036
IRASC5-040
IRASC5-041
IRASC5-042
IRASC5-043
IRASC5-044
IRASC5-046
IRASC5-047
IRASC5-050
IRASC5-051
IRASC5-052
IRASC5-053
IRASC5-054
IRASC5-056
IRASC5-057
IRASC5-058
IRASC5-059
IRASC5-060
IRASC5-061
IRASC5-062
IRASC5-063
IRASC5-064
IRASC5-065
IRASC5-067
IRASC5-068
IRASC5-069
IRASC5-071
IRASC5-072
IRASC5-073
IRASC5-074


In [126]:
standard_counts = df_s_temp['Document Type'].value_counts()
for standard, count in standard_counts.items():
    print(f"Standard: {standard}, Count: {count}")

Standard: Invoice, Count: 988
Standard: Credit Note, Count: 988


In [127]:
path_to_sample_xml_files

'/Users/cheongray/iras_api_make_xml/combine_all_copy/sample_xml_files'

In [128]:
# path_to_sample_xml_files = os.path.join(user_root,'sample_xml_files')

In [129]:
#paths for differetn base xmls
f_p_foreign_invoice_bis_peppol = path_to_sample_xml_files + "/sample_1.xml"


f_p_foreign_credit_bis_peppol =  path_to_sample_xml_files + '/sample_3.xml'

In [130]:
# f_p = '/Users/cheongray/iras_api_pytest_1/try_both/base_xml_files/foreign_invoice.xml'

In [131]:
def get_mapping_for_xml_invoice(file_path_of_sample_xml):

    namespace_map = {
        'sb': 'http://www.unece.org/cefact/namespaces/StandardBusinessDocumentHeader',
        'xs': 'http://www.w3.org/2001/XMLSchema',
        'cac': 'urn:oasis:names:specification:ubl:schema:xsd:CommonAggregateComponents-2',
        'cbc': 'urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2',
        'ubl': 'urn:oasis:names:specification:ubl:schema:xsd:Invoice-2',
        'xsi': "http://www.w3.org/2001/XMLSchema-instance",
        'xsd':"http://www.w3.org/2001/XMLSchema",
        'udt':"urn:un:unece:uncefact:data:specification:UnqualifiedDataTypesSchemaModule:2" ,
        'qdt':"urn:oasis:names:specification:ubl:schema:xsd:QualifiedDatatypes-2",
        'ext':"urn:oasis:names:specification:ubl:schema:xsd:CommonExtensionComponents-2",
        'ccts':"urn:un:unece:uncefact:documentation:2"
    #     'xmlns' : "urn:oasis:names:specification:ubl:schema:xsd:Invoice-2"
    }
    tree = etree.parse(file_path_of_sample_xml)
    target_root = tree.getroot()
    # format_key_map = []
    invoice_dict={}
    for element in target_root.iter():
        ancestors = [e.tag for e in element.iterancestors()][::-1] + [element.tag]

        string_ancestors = [e if isinstance(e, str) else str(e) for e in ancestors]

        path = ' / '.join(string_ancestors)
        for prefix, uri in namespace_map.items():
            path = path.replace('{' + uri + '}', prefix + ':')
        if 'sb:StandardBusinessDocument / ubl:Invoice / ' in path:
            path = path.replace('sb:StandardBusinessDocument / ', '')
            if element.attrib:
                xpath_base = path
                invoice_dict[xpath_base] = tree.getpath(element)
                for e,val in element.attrib.items():
                    xpath = xpath_base + ' / ' + '@' + e

                    invoice_dict[xpath] = tree.getpath(element)
            else:
                invoice_dict[path] = tree.getpath(element)

        else:
            if element.attrib:
                xpath_base = get_element_xpath(element)
                invoice_dict[xpath_base] = tree.getpath(element)
                for e,val in element.attrib.items():
                    xpath = xpath_base + ' / ' + '@' + e
                    invoice_dict[xpath] = tree.getpath(element)
            else:
                xpath = get_element_xpath(element)#defined on top
                invoice_dict[xpath] = tree.getpath(element)
    r_h = invoice_dict
    return r_h
def get_mapping_for_xml_credit(file_path_of_sample_xml):
    namespace_map = {
        'sb': 'http://www.unece.org/cefact/namespaces/StandardBusinessDocumentHeader',
        'xs': 'http://www.w3.org/2001/XMLSchema',
        'cac': 'urn:oasis:names:specification:ubl:schema:xsd:CommonAggregateComponents-2',
        'cbc': 'urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2',
        'ubl': 'urn:oasis:names:specification:ubl:schema:xsd:CreditNote-2',
        'xsi': "http://www.w3.org/2001/XMLSchema-instance",
        'xsd':"http://www.w3.org/2001/XMLSchema",
        'udt':"urn:un:unece:uncefact:data:specification:UnqualifiedDataTypesSchemaModule:2" ,
        'qdt':"urn:oasis:names:specification:ubl:schema:xsd:QualifiedDatatypes-2",
        'ext':"urn:oasis:names:specification:ubl:schema:xsd:CommonExtensionComponents-2",
        'ccts':"urn:un:unece:uncefact:documentation:2" 
    }
    tree = etree.parse(file_path_of_sample_xml)
    target_root = tree.getroot()
        # format_key_map = []
    credit_dict={}
    for element in target_root.iter():
        if isinstance(element, etree._Comment):
            continue

        else:
            ancestors = [e.tag for e in element.iterancestors()][::-1] + [element.tag]

            string_ancestors = [e if isinstance(e, str) else str(e) for e in ancestors]

            path = ' / '.join(string_ancestors)
            for prefix, uri in namespace_map.items():
                path = path.replace('{' + uri + '}', prefix + ':')

            if 'sb:StandardBusinessDocument / ubl:CreditNote / ' in path:

                path = path.replace('sb:StandardBusinessDocument / ', '')

                if element.attrib:
                    xpath_base = path
                    credit_dict[xpath_base] = tree.getpath(element)
                    for e,val in element.attrib.items():
                        xpath = xpath_base + ' / ' + '@' + e

                        credit_dict[xpath] = tree.getpath(element)
                else:
                    credit_dict[path] = tree.getpath(element)

            else:
                if element.attrib:
                    xpath_base = get_element_xpath(element)
                    credit_dict[xpath_base] = tree.getpath(element)
                    for e,val in element.attrib.items():
                        xpath = xpath_base + ' / ' + '@' + e
        #                 print(f'-->{xpath}')
                        credit_dict[xpath] = tree.getpath(element)
                else:
                    xpath = get_element_xpath(element)#defined on top
                    credit_dict[xpath] = tree.getpath(element)
    c_h = credit_dict
    return c_h
def credit_or_invoice(file_path):
#     print('********')
#     print(file_path)
    tree = etree.parse(file_path)
    target_root = tree.getroot()
    for index,e in enumerate(target_root.iter()):
        if index ==11:
            if 'Credit' in e.text:
                return 'credit'
            elif 'Invoice' in e.text:
                return 'invoice'
            else:
                return 'undefined'
def proper_mapping_final(file_path):

    if credit_or_invoice(file_path) == 'credit':
        r_h = get_mapping_for_xml_credit(file_path)
        return r_h
    elif credit_or_invoice(file_path) == 'invoice':
        r_h = get_mapping_for_xml_invoice(file_path)
        return r_h
    else:
        return None

In [132]:
def load_base_value_iden(file_path):
    if os.path.exists(file_path):
        
        # File exists, load the variable from the file
        if os.path.getsize(file_path) > 0:
                    # File is not empty, load the variable from the file
            with open(file_path, 'r') as file:
                
                variable_loaded = file.read().strip()
#                 print('loading')
#                 print(variable_loaded)
        else:
            # File exists but is empty, initialize with a default value
            variable_loaded = "11e1d6e4-74ba-49cc-9b56-2"
    else:
        # File does not exist, create the file and write the variable to it
#         print('+++++')
        variable_to_store = "11e1d6e4-74ba-49cc-9b56-2"
        with open(file_path, 'w') as file:
            file.write(variable_to_store)
        variable_loaded = variable_to_store
        
    return variable_loaded
def save_base_value_iden(file_path,variable_to_store):
    
    # Write the variable to the file
    with open(file_path, 'w') as file:
        file.write(variable_to_store)
def increment_value_iden(value,file_path):

    str_last = value.split('-')[-1]
    last= int(str_last)
    last+=1
    
    exclude_last = value.split('-')[:-1]
    temp = '-'.join(exclude_last)

    new_identifier = temp + '-' + str(last)
    
    if os.path.exists(file_path):
        with open(file_path, 'w') as file:
            variable_to_store = new_identifier
#             print('writing')
#             print(variable_to_store)
            file.write(variable_to_store)
    else:
        print('load base value first')

In [133]:
get_mapping_for_xml_credit(f_p_foreign_credit_bis_peppol)

{'StandardBusinessDocument': '/*',
 'StandardBusinessDocument / StandardBusinessDocumentHeader': '/*/*[1]',
 'StandardBusinessDocument / StandardBusinessDocumentHeader / HeaderVersion': '/*/*[1]/*[1]',
 'StandardBusinessDocument / StandardBusinessDocumentHeader / Sender': '/*/*[1]/*[2]',
 'StandardBusinessDocument / StandardBusinessDocumentHeader / Sender / Identifier': '/*/*[1]/*[2]/*',
 'StandardBusinessDocument / StandardBusinessDocumentHeader / Sender / Identifier / @Authority': '/*/*[1]/*[2]/*',
 'StandardBusinessDocument / StandardBusinessDocumentHeader / Receiver': '/*/*[1]/*[3]',
 'StandardBusinessDocument / StandardBusinessDocumentHeader / Receiver / Identifier': '/*/*[1]/*[3]/*',
 'StandardBusinessDocument / StandardBusinessDocumentHeader / Receiver / Identifier / @Authority': '/*/*[1]/*[3]/*',
 'StandardBusinessDocument / StandardBusinessDocumentHeader / DocumentIdentification': '/*/*[1]/*[4]',
 'StandardBusinessDocument / StandardBusinessDocumentHeader / DocumentIdentificat

In [134]:
# 11e1d6e4-74ba-49cc-9b56-2

# filepath_base_identifier = '/Users/cheongray/iras_api_pytest_1/stored_base_identifier.txt'
# a=load_base_value_iden(filepath_base_identifier)
# increment_value_iden(a,filepath_base_identifier)

In [135]:
# save_base_value_iden(filepath_base_identifier,'11e1d6e4-74ba-49cc-9b56-2')

In [136]:
namespace_map = {
    'sb': 'http://www.unece.org/cefact/namespaces/StandardBusinessDocumentHeader',
    'xs': 'http://www.w3.org/2001/XMLSchema',
    'cac': 'urn:oasis:names:specification:ubl:schema:xsd:CommonAggregateComponents-2',
    'cbc': 'urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2',
    }


In [137]:
f_p_foreign_credit_bis_peppol

'/Users/cheongray/iras_api_make_xml/combine_all_copy/sample_xml_files/sample_3.xml'

In [138]:
tree_foreign_credit_bis_peppol = etree.parse(f_p_foreign_credit_bis_peppol)
target_root_foreign_credit_bis_peppol = tree_foreign_credit_bis_peppol.getroot()

for index,element in enumerate(target_root_foreign_credit_bis_peppol.iter()):
    print(tree_foreign_credit_bis_peppol.getpath(element))

/*
/*/*[1]
/*/*[1]/*[1]
/*/*[1]/*[2]
/*/*[1]/*[2]/*
/*/*[1]/*[3]
/*/*[1]/*[3]/*
/*/*[1]/*[4]
/*/*[1]/*[4]/*[1]
/*/*[1]/*[4]/*[2]
/*/*[1]/*[4]/*[3]
/*/*[1]/*[4]/*[4]
/*/*[1]/*[4]/*[5]
/*/*[1]/*[5]
/*/*[1]/*[5]/*[1]
/*/*[1]/*[5]/*[1]/*[1]
/*/*[1]/*[5]/*[1]/*[2]
/*/*[1]/*[5]/*[1]/*[3]
/*/*[1]/*[5]/*[2]
/*/*[1]/*[5]/*[2]/*[1]
/*/*[1]/*[5]/*[2]/*[2]
/*/*[1]/*[5]/*[2]/*[3]
/*/*[1]/*[5]/*[3]
/*/*[1]/*[5]/*[3]/*[1]
/*/*[1]/*[5]/*[3]/*[2]
/*/*[1]/*[5]/*[4]
/*/*[1]/*[5]/*[4]/*[1]
/*/*[1]/*[5]/*[4]/*[2]
/*/*[2]
/*/*[2]/comment()[1]
/*/*[2]/cbc:UBLVersionID
/*/*[2]/cbc:CustomizationID
/*/*[2]/comment()[2]
/*/*[2]/cbc:ProfileID
/*/*[2]/comment()[3]
/*/*[2]/cbc:ID
/*/*[2]/comment()[4]
/*/*[2]/cbc:UUID
/*/*[2]/cbc:IssueDate
/*/*[2]/comment()[5]
/*/*[2]/cbc:CreditNoteTypeCode
/*/*[2]/comment()[6]
/*/*[2]/cbc:Note
/*/*[2]/comment()[7]
/*/*[2]/cbc:DocumentCurrencyCode
/*/*[2]/cbc:AccountingCost
/*/*[2]/comment()[8]
/*/*[2]/comment()[9]
/*/*[2]/cbc:BuyerReference
/*/*[2]/cac:InvoicePeriod
/*/*[2]/cac:Inv

In [139]:
target_root_foreign_credit_bis_peppol.xpath('/*/*[2]/cbc:CustomizationID',namespaces=namespace_map)[0].text

'urn:peppol:pint:billing-1@sg-1:LocalTaxInvoice:sg:1.0'

In [140]:
#Invoice credit samples(dont need)
#missing values front back of element make sure there
#instance identifier chnage sender identifier for invalid and missing values
#creatioindate and time must be in correct format(dont need)
#suplierparty and customer endpoint scheme id (all the attributes)
#for field does not exist the whole element gone only need attribute gone(attribute and valeu should both be gone only element remain)

In [141]:
# save_base_value('0195:SGUEN20',file_path_sender_identifier)

In [142]:
# # load_base_value(file_path_sender_identifier)
# increment_value(load_base_value(file_path_sender_identifier),file_path_sender_identifier)

In [143]:
filepath_base_identifier = os.path.join(user_root,'instance_identifier.txt')
file_path_sender_identifier = os.path.join(user_root,'sender_iden_store.txt')


glob_check_correct= []
def adjust_identifier(root_copy):
    base_identifier = load_base_value_iden(filepath_base_identifier)

    for index,e in enumerate(root_copy.iter()):
        if index == 10:#rsender identifier:
            e.text = base_identifier
            increment_value_iden(base_identifier,filepath_base_identifier)
    return root_copy

def adjust_sender_identifier(root_copy):
    base_identifier = load_base_value(file_path_sender_identifier)

    for index,e in enumerate(root_copy.iter()):
        if index == 4:#rsender identifier:
            e.text = base_identifier
            increment_value(base_identifier,file_path_sender_identifier)
    return root_copy
                        
def adjust_customisation_id(root_copy,value):
     for index,e in enumerate(root_copy.iter()):
        if index == 31:
            e.text = value

            
        
def adjust_profile_id(root_copy,value):
    for index,e in enumerate(root_copy.iter()):
        if index == 33:
            e.text = value
            

def process_xpath_mod(r_h, xpath_find, target_root, attrib_yes, filename, c, field_value,descript,special_c5_003,biz_pint,standard,rule_id,test_case_id):
    namespace_map = {
    'sb': 'http://www.unece.org/cefact/namespaces/StandardBusinessDocumentHeader',
    'xs': 'http://www.w3.org/2001/XMLSchema',
    'cac': 'urn:oasis:names:specification:ubl:schema:xsd:CommonAggregateComponents-2',
    'cbc': 'urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2',
    }
    # Create a deep copy of the original target_root tree
    original_tree_copy = copy.deepcopy(target_root)
    original_tree_copy = adjust_identifier(original_tree_copy)
    if special_c5_003:
        if 'invalid' in descript or 'missing' in descript:
            print('CHANGING BACK SINCE....', descript)
            special_c5_003 = False
            #adjust sender identifier for these 2 special cases
            print('adjust sender identifer for special case..')
            adjust_sender_identifier(original_tree_copy)


    if standard == 'PEPPOL':
        if biz_pint == 'BIS':
            value = 'urn:fdc:peppol.eu:2017:poacc:billing:01:1.0'
            adjust_profile_id(original_tree_copy,value)
            value_cus = 'urn:cen.eu:en16931:2017#conformant#urn:fdc:peppol.eu:2017:poacc:billing:international:sg:3.0'
            adjust_customisation_id(original_tree_copy,value_cus)

        if biz_pint == 'PINT':
            value = 'urn:peppol:bis:billing'
            adjust_profile_id(original_tree_copy,value)
            value_cus = 'urn:peppol:pint:billing-1@sg-1'
            adjust_customisation_id(original_tree_copy,value_cus)

            
    if standard == 'Non-PEPPOL':
        if biz_pint == 'BIS':
            value = 'urn:fdc:peppol.eu:2017:poacc:billing:01:1.0'
            adjust_profile_id(original_tree_copy,value)
            value_cus = 'urn:cen.eu:en16931:2017#conformant#urn:fdc:peppol.eu:2017:poacc:billing:LocalTaxInvoice:sg:1.0'
            adjust_customisation_id(original_tree_copy,value_cus)

        if biz_pint == 'PINT':
            value = 'urn:peppol:bis:billing'
            adjust_profile_id(original_tree_copy,value)
            value_cus = 'urn:peppol:pint:billing-1@sg-1:LocalTaxInvoice:sg:1.0'
            adjust_customisation_id(original_tree_copy,value_cus)

    
    
#     if special_c5_003:
#         adjust_sender_identifier(original_tree_copy)
    if xpath_find in r_h:
        xpath_target = r_h[xpath_find]
        element_temp = original_tree_copy.xpath(xpath_target, namespaces=namespace_map)
        if len(element_temp)>0:
            element = element_temp[0]
        elif len(element_temp)<=0:
            print(f'---M---error making file{filename} descript:{descript} eleemt {element_temp} xpath find {xpath_find}')
            return c
        insert_val = str(field_value)
        print(f'old text {element.text}')
        
        if insert_val == 'nan' and  descript == 'Field exists with missing value' and not attrib_yes:#so if invalid value Nan which is read as nan type will still be assigned their invlaid value
                insert_val = ''
        elif insert_val == 'nan' and  descript == 'Field exists with invalid value':
                insert_val = 'NA'
                print(f'changing for element {element} xpath_target={xpath_target } insert_val = {insert_val}')
        else:
            if descript == 'Field exists with valid value' and xpath_find == 'StandardBusinessDocument / StandardBusinessDocumentHeader / DocumentIdentification / InstanceIdentifier':
                pass
        if not attrib_yes:#no attribute
            print(f'changing element text for { xpath_find} element {element} xpath_target={xpath_target }')
            print(f'insert val is {insert_val}')
            element.text = insert_val
            print(f'new text {element.text}')
            
            if special_c5_003:
                    print('adjusting for special case modify')
                    original_tree_copy = adjust_identifier(original_tree_copy)
            tree = etree.ElementTree(original_tree_copy)
            
            indexed_filename =rule_id +'_' + 'Test_case_' + str(test_case_id) + '_' + sheet_name + '.xml'
            filepath = os.path.join(folder_name, indexed_filename)
            tree.write(filepath, encoding='utf-8', xml_declaration=True)
            
            
#             print(indexed_filename , c)
            
            c += 1
            return c
        else:
#             print(f'old text[2] {element.text}')
            key_attrib = xpath_find.split('@')[-1]
            print(f'modifying attribute for { xpath_find} element {element} xpath_target={xpath_target }key attrib = {key_attrib} insert_val = {insert_val}')
            print('  ')
            if insert_val == 'nan':
           
                element.attrib[key_attrib] =  ''
#                 print(f'new text {element.text}')

                tree = etree.ElementTree(original_tree_copy)
                if special_c5_003:
                    original_tree_copy = adjust_identifier(original_tree_copy)
                indexed_filename =rule_id +'_' + 'Test_case_' + str(test_case_id) + '_' + sheet_name + '.xml'
                filepath = os.path.join(folder_name, indexed_filename)
                tree.write(filepath, encoding='utf-8', xml_declaration=True)
                c+=1
                return c
            else:
                print(f'old value {element.attrib} ')
                
                element.attrib[key_attrib] = insert_val
                print(f'new value {element.attrib} ')
                if special_c5_003:
                    print('adjusting....')
                    original_tree_copy = adjust_identifier(original_tree_copy)
                tree = etree.ElementTree(original_tree_copy)
#                 indexed_filename = filename + '_FileNumber_' + str(c) + '_m_' + '.xml'
                indexed_filename =rule_id +'_' + 'Test_case_' + str(test_case_id) + '_' + sheet_name + '.xml'
                filepath = os.path.join(folder_name, indexed_filename)
                tree.write(filepath, encoding='utf-8', xml_declaration=True)
                c+=1
                return c
#             print(indexed_filename , c)
            
    else:
        print(f'key not found for {xpath_find}')
    
        return c
def process_xpath_remove(r_h, xpath_find, target_root, attrib_yes,filename,c,special_c5_003,biz_pint,standard,rule_id,test_case_id,descript):#clear up code..too messy
    namespace_map = {
    'sb': 'http://www.unece.org/cefact/namespaces/StandardBusinessDocumentHeader',
    'xs': 'http://www.w3.org/2001/XMLSchema',
    'cac': 'urn:oasis:names:specification:ubl:schema:xsd:CommonAggregateComponents-2',
    'cbc': 'urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2',
    }
    print(f'RULE ID IS {rule_id}')
    original_tree_copy = copy.deepcopy(target_root)
    original_tree_copy = adjust_identifier(original_tree_copy)
    if special_c5_003:
        if 'invalid' in descript or 'missing' in descript:
            print('CHANGING BACK SINCE....',descript)
            special_c5_003 = False
    if standard == 'PEPPOL':
        if biz_pint == 'BIS':
            value = 'urn:fdc:peppol.eu:2017:poacc:billing:01:1.0'
            adjust_profile_id(original_tree_copy,value)
            value_cus = 'urn:cen.eu:en16931:2017#conformant#urn:fdc:peppol.eu:2017:poacc:billing:international:sg:3.0'
            adjust_customisation_id(original_tree_copy,value_cus)

        if biz_pint == 'PINT':
            value = 'urn:peppol:bis:billing'
            adjust_profile_id(original_tree_copy,value)
            value_cus = 'urn:peppol:pint:billing-1@sg-1'
            adjust_customisation_id(original_tree_copy,value_cus)

            
    if standard == 'Non-PEPPOL':
        if biz_pint == 'BIS':
            value = 'urn:fdc:peppol.eu:2017:poacc:billing:01:1.0'
            adjust_profile_id(original_tree_copy,value)
            value_cus = 'urn:cen.eu:en16931:2017#conformant#urn:fdc:peppol.eu:2017:poacc:billing:LocalTaxInvoice:sg:1.0'
            adjust_customisation_id(original_tree_copy,value_cus)

        if biz_pint == 'PINT':
            value = 'urn:peppol:bis:billing'
            adjust_profile_id(original_tree_copy,value)
            value_cus = 'urn:peppol:pint:billing-1@sg-1:LocalTaxInvoice:sg:1.0'
            adjust_customisation_id(original_tree_copy,value_cus)
    
    if xpath_find in r_h:
        print('REMOVING ELEMENT..')
        xpath_target = r_h[xpath_find]
      
        element_temp = original_tree_copy.xpath(xpath_target, namespaces=namespace_map)
        
       
    
        
        
        if len(element_temp)>0:
            element = element_temp[0]
        elif len(element_temp)<=0:
            print(f'--R--error making file{filename} descript:{descript} eleemt {element_temp} xpath find {xpath_find}')
            return c
        
        if attrib_yes:
            key_attrib = xpath_find.split('@')[-1]
            print(f'removing attrib {xpath_find}')
            del element.attrib[key_attrib]
            if special_c5_003:
                    original_tree_copy = adjust_identifier(original_tree_copy)
            tree = etree.ElementTree(original_tree_copy)
            indexed_filename =rule_id +'_' + 'Test_case_' + str(test_case_id) + '_' + sheet_name + '.xml'
            filepath = os.path.join(folder_name,indexed_filename)

            tree.write(filepath, encoding='utf-8', xml_declaration=True)
            c+=1
            return c
            
     

        parent = element.getparent()
        print('parnet created')
        if parent is not None:
            print('element parent is not none')
            parent.remove(element)

            new_tree = etree.ElementTree(original_tree_copy)
            if special_c5_003:
                    original_tree_copy = adjust_identifier(original_tree_copy)
            
            indexed_filename =rule_id +'_' + 'Test_case_' + str(test_case_id) + '_' + sheet_name + '.xml'
            filepath = os.path.join(folder_name,indexed_filename)
            new_tree.write(filepath, encoding='utf-8', xml_declaration=True)
#             print(indexed_filename , c)
            c+=1
            return c

    return c

In [144]:
function_mapping = {
    'Field exists with valid value': 0,
    'Field exists with invalid value': 0,
    'Field exists with missing value':0,
    'Field does not exist': 1,
    'Field exists with valid positive value': 0,
    'Field exists with valid negative value': 0,
    'Field exists with zero value': 0
}
a=df_s_temp['Test Case Description'].unique().tolist()
a
for key in a:
    if key not in function_mapping:
        function_mapping[key] = 0


In [145]:
r_h = get_mapping_for_xml_invoice(f_p_foreign_invoice_bis_peppol)
c_h = get_mapping_for_xml_credit(f_p_foreign_credit_bis_peppol)
r_h.update(c_h)

In [146]:
# root= etree.parse(f_p_foreign_credit_bis_peppol)
# for index,e in enumerate(root.iter()):
#     if index == 4:
#         print(e.text)
# root= etree.parse(f_p_foreign_invoice_bis_peppol)
# for index,e in enumerate(root.iter()):
#     if index == 6:
#         print(e.text)

In [147]:
#MAKE folder to store xml files
folder_name = output_folder + '/xml_files_for'+'_'+sheet_name



if not os.path.exists(folder_name):
    os.makedirs(folder_name)
if os.path.exists(folder_name):
    shutil.rmtree(folder_name)
    os.makedirs(folder_name)


# f_p = r"/Users/cheongray/iras_api_pytest_1/base_correct_mine.xml"



tree_foreign_invoice_bis_peppol = etree.parse(f_p_foreign_invoice_bis_peppol)
target_root_foreign_invoice_bis_peppol = tree_foreign_invoice_bis_peppol.getroot()



tree_foreign_credit_bis_peppol = etree.parse(f_p_foreign_credit_bis_peppol)
target_root_foreign_credit_bis_peppol = tree_foreign_credit_bis_peppol.getroot()

c=1
attrib_yes=False
num_files_with_nan_descript = 0
num_error_files = 0


special_c5_003 = False
for test_case_key in result_dict:
    print('                    ')
    print('||||||||||||START||||||||||||||||||||||||')
    temp_1 = result_dict[test_case_key]#Test Case Key: IRASC5-001
    print(f'test case key = {test_case_key}')
    if test_case_key == 'IRASC5-003':
        print('+++++++++++++++')
        print('+++++++++++++++')
        print('special case IRASC5-003')
        print('+++++++++++++++')
        print('+++++++++++++++')
        print('   ')
        special_c5_003 = True
        
    elif test_case_key != 'IRASC5-003':
        special_c5_003 = False#lock back
    for index,test_cases in enumerate(temp_1):
        
        attrib_yes = False#set back to false

        key = next(iter(test_cases)) #1.0
        
        xpath_find = test_cases[key]['Field'][0].strip()#XPath Find: StandardBusinessDocument / StandardBusinessDocumentHeader / Sender / Identifier
        
#         print(xpath_find)
        descript =  test_cases[key]['Test Case Description'][0]# Description: Field exists with valid value
#         #special case for IRASC5-003 ,if its 'invalid value' and 'missing value',leave it as it 


        test_case_id = test_cases[key]['Test Case ID']
 
        field_value = test_cases[key]['Field Value'][0]# Field Value: 0195:SGUEN200012345A
        
        document_type = test_cases[key]['Document Type'][0]
        
        biz_pint = test_cases[key]['BIS/PINT'][0]
        
        rule_id = test_cases[key]['Rule ID'][0]
        standard = test_cases[key]['Standard'][0]
        
        sample = test_cases[key]['Sample'][0]
        
#         print(f"Document Type: {document_type}\nBIS/PIN: {biz_pint}\nStandard: {standard}")

        if sample == 'Sample 1':
            target_root = target_root_foreign_invoice_bis_peppol

        if sample == 'Sample 3':
            target_root = target_root_foreign_credit_bis_peppol
        
    
            
    

        if '@' in xpath_find:
            attrib_yes = True
        if descript in function_mapping:
            choice = function_mapping[descript]
            if choice == 0:#modify element
#                 print(f'zero:{c}')
                filename = test_case_key + '_Test_case_' + str(int(key))
                old_c = c
                c = process_xpath_mod(r_h, xpath_find, target_root, attrib_yes,filename,c,field_value,descript,special_c5_003, biz_pint,standard,rule_id,int(test_case_id[0]))
                if c == old_c:
                    num_error_files +=1 #just a checker to check element has been modified
            elif choice == 1:#remove element
                print(f't_id = {key}')
                filename = test_case_key + '_Test_case_' + str(int(key))
                old_c = c
                c= process_xpath_remove(r_h, xpath_find, target_root, attrib_yes,filename,c,special_c5_003, biz_pint,standard,rule_id,int(test_case_id[0]),descript)
                if c == old_c:
                    num_error_files +=1
            else:
                print(f'not doing test case{key} with path:{xpath_find}')
        else:
            num_files_with_nan_descript += 1

            print(f'function descript not in mapping {descript} test case id {key}')

            

print(f'num_files_with_nan_descript {num_files_with_nan_descript}') 
print(f'{c-1} files  created')
final_iden_value = load_base_value_iden(filepath_base_identifier)
print(f'final base value :',final_iden_value)
print(f'failed to make this {num_error_files} number of files')

                
                

                    
||||||||||||START||||||||||||||||||||||||
test case key = IRASC5-001
old text 0195:SGUENEYSIT1011
changing element text for StandardBusinessDocument / StandardBusinessDocumentHeader / Sender / Identifier element <Element {http://www.unece.org/cefact/namespaces/StandardBusinessDocumentHeader}Identifier at 0x153d2bdc0> xpath_target=/*/*[1]/*[2]/*
insert val is 0195:SGUEN200012345A
new text 0195:SGUEN200012345A
old text 0195:SGUENEYSIT1011
changing element text for StandardBusinessDocument / StandardBusinessDocumentHeader / Sender / Identifier element <Element {http://www.unece.org/cefact/namespaces/StandardBusinessDocumentHeader}Identifier at 0x156ab1cc0> xpath_target=/*/*[1]/*[2]/*
insert val is 0195:SGUEN
new text 0195:SGUEN
old text 0195:SGUENEYSIT1011
changing element text for StandardBusinessDocument / StandardBusinessDocumentHeader / Sender / Identifier element <Element {http://www.unece.org/cefact/namespaces/StandardBusinessDocumentHeader}Identifier at 0x15087

old text 2020-05-27
changing element text for ubl:CreditNote / cbc:IssueDate element <Element {urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2}IssueDate at 0x156b0f180> xpath_target=/*/*[2]/cbc:IssueDate
insert val is 12 Aug 2024
new text 12 Aug 2024
old text 2020-05-27
changing element text for ubl:CreditNote / cbc:IssueDate element <Element {urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2}IssueDate at 0x153e786c0> xpath_target=/*/*[2]/cbc:IssueDate
insert val is 
new text 
t_id = 288.0
RULE ID IS IRASC5-009
REMOVING ELEMENT..
parnet created
element parent is not none
                    
||||||||||||START||||||||||||||||||||||||
test case key = IRASC5-010
old text 380
changing element text for ubl:Invoice / cbc:InvoiceTypeCode element <Element {urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2}InvoiceTypeCode at 0x1536c40c0> xpath_target=/*/*[2]/cbc:InvoiceTypeCode
insert val is 380
new text 380
old text 380
changing elemen

old text Gallery Photo Supplier
changing for element <Element {urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2}RegistrationName at 0x1536642c0> xpath_target=/*/*[2]/cac:AccountingSupplierParty/cac:Party/cac:PartyLegalEntity/cbc:RegistrationName insert_val = NA
changing element text for ubl:CreditNote / cac:AccountingSupplierParty / cac:Party / cac:PartyLegalEntity / cbc:RegistrationName element <Element {urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2}RegistrationName at 0x1536642c0> xpath_target=/*/*[2]/cac:AccountingSupplierParty/cac:Party/cac:PartyLegalEntity/cbc:RegistrationName
insert val is NA
new text NA
old text Gallery Photo Supplier
changing element text for ubl:CreditNote / cac:AccountingSupplierParty / cac:Party / cac:PartyLegalEntity / cbc:RegistrationName element <Element {urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2}RegistrationName at 0x156b091c0> xpath_target=/*/*[2]/cac:AccountingSupplierParty/cac:Party

old text 2374.05
changing element text for ubl:Invoice / cac:TaxTotal / cbc:TaxAmount element <Element {urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2}TaxAmount at 0x15691d280> xpath_target=/*/*[2]/cac:TaxTotal/cbc:TaxAmount
insert val is -10000000.6
new text -10000000.6
old text 2374.05
changing element text for ubl:Invoice / cac:TaxTotal / cbc:TaxAmount element <Element {urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2}TaxAmount at 0x153bb6b00> xpath_target=/*/*[2]/cac:TaxTotal/cbc:TaxAmount
insert val is 0
new text 0
old text 2374.05
changing for element <Element {urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2}TaxAmount at 0x154c99a00> xpath_target=/*/*[2]/cac:TaxTotal/cbc:TaxAmount insert_val = NA
changing element text for ubl:Invoice / cac:TaxTotal / cbc:TaxAmount element <Element {urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2}TaxAmount at 0x154c99a00> xpath_target=/*/*[2]/cac:TaxTotal/cbc:TaxAm

old text SR
changing element text for ubl:CreditNote / cac:TaxTotal / cac:TaxSubtotal / cac:TaxCategory / cbc:ID element <Element {urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2}ID at 0x154c99a00> xpath_target=/*/*[2]/cac:TaxTotal/cac:TaxSubtotal/cac:TaxCategory/cbc:ID
insert val is ABC1
new text ABC1
old text SR
changing element text for ubl:CreditNote / cac:TaxTotal / cac:TaxSubtotal / cac:TaxCategory / cbc:ID element <Element {urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2}ID at 0x153b375c0> xpath_target=/*/*[2]/cac:TaxTotal/cac:TaxSubtotal/cac:TaxCategory/cbc:ID
insert val is 
new text 
t_id = 1100.0
RULE ID IS IRASC5-044
REMOVING ELEMENT..
parnet created
element parent is not none
old text SR
changing element text for ubl:CreditNote / cac:TaxTotal / cac:TaxSubtotal / cac:TaxCategory / cbc:ID element <Element {urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2}ID at 0x1569c5280> xpath_target=/*/*[2]/cac:TaxTotal/cac:TaxS

old text 36289.05
changing for element <Element {urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2}TaxInclusiveAmount at 0x153b375c0> xpath_target=/*/*[2]/cac:LegalMonetaryTotal/cbc:TaxInclusiveAmount insert_val = NA
changing element text for ubl:CreditNote / cac:LegalMonetaryTotal / cbc:TaxInclusiveAmount element <Element {urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2}TaxInclusiveAmount at 0x153b375c0> xpath_target=/*/*[2]/cac:LegalMonetaryTotal/cbc:TaxInclusiveAmount
insert val is NA
new text NA
old text 36289.05
changing element text for ubl:CreditNote / cac:LegalMonetaryTotal / cbc:TaxInclusiveAmount element <Element {urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2}TaxInclusiveAmount at 0x153432740> xpath_target=/*/*[2]/cac:LegalMonetaryTotal/cbc:TaxInclusiveAmount
insert val is 
new text 
t_id = 1358.0
RULE ID IS IRASC5-054
REMOVING ELEMENT..
parnet created
element parent is not none
old text 36289.05
changing element 

old text 10
modifying attribute for ubl:CreditNote / cac:CreditNoteLine / cbc:CreditedQuantity / @unitCode element <Element {urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2}CreditedQuantity at 0x153c5e640> xpath_target=/*/*[2]/cac:CreditNoteLine/cbc:CreditedQuantitykey attrib = unitCode insert_val = KG
  
old value {'unitCode': 'H87'} 
new value {'unitCode': 'KG'} 
old text 10
modifying attribute for ubl:CreditNote / cac:CreditNoteLine / cbc:CreditedQuantity / @unitCode element <Element {urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2}CreditedQuantity at 0x15481a740> xpath_target=/*/*[2]/cac:CreditNoteLine/cbc:CreditedQuantitykey attrib = unitCode insert_val = 100
  
old value {'unitCode': 'H87'} 
new value {'unitCode': '100'} 
old text 10
modifying attribute for ubl:CreditNote / cac:CreditNoteLine / cbc:CreditedQuantity / @unitCode element <Element {urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2}CreditedQuantity at 0x153c

t_id = 1818.0
RULE ID IS IRASC5-069
REMOVING ELEMENT..
parnet created
element parent is not none
old text 1000.00
changing element text for ubl:Invoice / cac:InvoiceLine / cac:Price / cbc:PriceAmount element <Element {urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2}PriceAmount at 0x153cbf880> xpath_target=/*/*[2]/cac:InvoiceLine[2]/cac:Price/cbc:PriceAmount
insert val is 15000000.15
new text 15000000.15
old text 1000.00
changing element text for ubl:Invoice / cac:InvoiceLine / cac:Price / cbc:PriceAmount element <Element {urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2}PriceAmount at 0x150737040> xpath_target=/*/*[2]/cac:InvoiceLine[2]/cac:Price/cbc:PriceAmount
insert val is -10000000.6
new text -10000000.6
old text 1000.00
changing element text for ubl:Invoice / cac:InvoiceLine / cac:Price / cbc:PriceAmount element <Element {urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2}PriceAmount at 0x154ca88c0> xpath_target=/*/*[2]/ca

In [102]:
# work_directory = '/Users/cheongray/iras_api_clean_1/1_Input_Validation'
# xml_file_path = work_directory + "/xml_files_for_1_Input Validation"
# filename_for_csv_with_results = '1_input_testing_csv_results'
# filename_for_csv_with_results_full_path  = work_directory + '/test_results/'+ filename_for_csv_with_results + '.csv'
# filename_for_csv_with_results_full_path

In [103]:
!pip freeze > requirements.txt